In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

# 1. Scrape `go2senkyo.com` for candidate information

## 1.1. Assembly election candidates

In [9]:
a_url = "https://go2senkyo.com/local/senkyo/21754"
a_response = requests.get(a_url)
a_soup = BeautifulSoup(a_response.content, "html.parser")

In [10]:
a_candidates = []
for candidate in a_soup.find_all("h2", class_="m_senkyo_result_data_ttl"):
    dict = {}
    dict['name'] = candidate.find("a").text
    dict['katakana'] = candidate.find("a").find("span").text
    a_candidates.append(dict)

In [11]:
candidate_num = 0
for candidate_demo in a_soup.find_all("div", class_="m_senkyo_result_data_bottom_right"):
    print(f"candidate {candidate_num}")
    index = 0
    for info in candidate_demo.find("p", class_="m_senkyo_result_data_para").find_all("span"):
        if index == 0:
            a_candidates[candidate_num]['age_sex'] = info.text
            index += 1
        if index == 1:
            a_candidates[candidate_num]['new_old'] = info.text
    candidate_num += 1

candidate 0
candidate 1
candidate 2
candidate 3
candidate 4
candidate 5
candidate 6
candidate 7
candidate 8
candidate 9
candidate 10
candidate 11
candidate 12
candidate 13
candidate 14
candidate 15
candidate 16
candidate 17
candidate 18
candidate 19
candidate 20
candidate 21
candidate 22
candidate 23
candidate 24
candidate 25
candidate 26
candidate 27
candidate 28
candidate 29
candidate 30
candidate 31
candidate 32
candidate 33
candidate 34
candidate 35
candidate 36
candidate 37
candidate 38
candidate 39
candidate 40
candidate 41
candidate 42
candidate 43
candidate 44
candidate 45
candidate 46
candidate 47
candidate 48
candidate 49
candidate 50
candidate 51
candidate 52
candidate 53
candidate 54
candidate 55


In [12]:
candidate_num = 0
for candidate_party in a_soup.find_all("div", class_="m_senkyo_result_data_bottom_left"):
    a_candidates[candidate_num]['party'] = candidate_party.find("p", class_="m_senkyo_result_data_circle").text
    candidate_num += 1

In [13]:
candidate_num = 0
for candidate_work in a_soup.find_all("div", class_="m_senkyo_result_data_bottom_right"):
    work = candidate_work.find("p", class_="m_senkyo_result_data_para small").text
    if work:
        a_candidates[candidate_num]['vocation'] = work
    else:
        a_candidates[candidate_num]['vocation'] = "NA"
    candidate_num += 1

In [14]:
a_candidates

[{'name': '\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t金井 たかしカナイ タカシ\n',
  'katakana': 'カナイ タカシ',
  'age_sex': '60歳  (男) ',
  'new_old': '新人',
  'party': '自由民主党',
  'vocation': '弁護士'},
 {'name': '\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t丸山 れいこマルヤマ レイコ\n',
  'katakana': 'マルヤマ レイコ',
  'age_sex': '70歳  (女) ',
  'new_old': '新人',
  'party': '日本維新の会',
  'vocation': '政治家秘書'},
 {'name': '\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t太田 あつしオオタ アツシ\n',
  'katakana': 'オオタ アツシ',
  'age_sex': '48歳  (男) ',
  'new_old': '新人',
  'party': '無所属',
  'vocation': '会社員'},
 {'name': '\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t福本 みつひろフクモト ミツヒロ\n',
  'katakana': 'フクモト ミツヒロ',
  'age_sex': '51歳  (男) ',
  'new_old': '現職',
  'party': '自由民主党',
  'vocation': '江戸川区議会議長'},
 {'name': '\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t岩田 まさかずイワタ マサカズ\n',
  'katakana': 'イワタ マサカズ',
  'age_sex': '49歳  (男) ',
  'new_old': '現職',
  'party': '無所属',
  'vocation': '江戸川区議会・区議会江戸川クラブ（幹事長）'},
 {'name': '\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t勝山 まゆみカツヤマ マユミ\n',
  'katakana': 'カツヤマ マユミ',
  '

In [15]:
a_candidates_df = pd.DataFrame(a_candidates)
a_candidates_df['category'] = 'A'

In [16]:
a_candidates_df

,name,katakana,age_sex,new_old,party,vocation,category
0,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t金井 たかしカナイ タカシ\n,カナイ タカシ,60歳 (男),新人,自由民主党,弁護士,A
1,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t丸山 れいこマルヤマ レイコ\n,マルヤマ レイコ,70歳 (女),新人,日本維新の会,政治家秘書,A
2,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t太田 あつしオオタ アツシ\n,オオタ アツシ,48歳 (男),新人,無所属,会社員,A
3,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t福本 みつひろフクモト ミツヒロ\n,フクモト ミツヒロ,51歳 (男),現職,自由民主党,江戸川区議会議長,A
4,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t岩田 まさかずイワタ マサカズ\n,イワタ マサカズ,49歳 (男),現職,無所属,江戸川区議会・区議会江戸川クラブ（幹事長）,A
5,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t勝山 まゆみカツヤマ マユミ\n,カツヤマ マユミ,54歳 (女),新人,自由民主党,自由民主党東京都江戸川区第十一支部長,A
6,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t五十嵐 まさおイガラシ マサオ\n,イガラシ マサオ,38歳 (男),新人,参政党,NA,A
7,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t肌野 ゆうじハダノ ユウジ\n,ハダノ ユウジ,53歳 (男),新人,自由民主党,会社員,A
8,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t間宮 由美マミヤ ユミ\n,マミヤ ユミ,60歳 (女),現職,無所属,江戸川区議会議員,A
9,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tしかくら 勇シカクラ イサム\n,シカクラ イサム,58歳 (男),現職,自由民主党,江戸川区議会自民党幹事,A


## 1.2 Mayor election candidates

In [17]:
m_url = "https://go2senkyo.com/local/senkyo/21753"
m_response = requests.get(m_url)
m_soup = BeautifulSoup(m_response.content, "html.parser")

In [18]:
m_candidates = []
for candidate in m_soup.find_all("h2", class_="m_senkyo_result_data_ttl"):
    dict = {}
    dict['name'] = candidate.find("a").text
    dict['katakana'] = candidate.find("a").find("span").text
    m_candidates.append(dict)

In [19]:
candidate_num = 0
for candidate_demo in m_soup.find_all("div", class_="m_senkyo_result_data_bottom_right"):
    print(f"candidate {candidate_num}")
    index = 0
    for info in candidate_demo.find("p", class_="m_senkyo_result_data_para").find_all("span"):
        if index == 0:
            m_candidates[candidate_num]['age_sex'] = info.text
            index += 1
        if index == 1:
            m_candidates[candidate_num]['new_old'] = info.text
    candidate_num += 1

candidate 0
candidate 1


In [20]:
candidate_num = 0
for candidate_party in m_soup.find_all("div", class_="m_senkyo_result_data_bottom_left"):
    m_candidates[candidate_num]['party'] = candidate_party.find("p", class_="m_senkyo_result_data_circle").text
    candidate_num += 1

In [21]:
candidate_num = 0
for candidate_work in m_soup.find_all("div", class_="m_senkyo_result_data_bottom_right"):
    work = candidate_work.find("p", class_="m_senkyo_result_data_para small").text
    if work:
        m_candidates[candidate_num]['vocation'] = work
    else:
        m_candidates[candidate_num]['vocation'] = "NA"
    candidate_num += 1

In [22]:
m_candidates_df = pd.DataFrame(m_candidates)
m_candidates_df['category'] = "M"

In [23]:
m_candidates_df

,name,katakana,age_sex,new_old,party,vocation,category
0,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t斉藤 たけしサイトウ タケシ\n,サイトウ タケシ,60歳 (男),現職,無所属,江戸川区長,M
1,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tセバタ 勇セバタ イサム\n,セバタ イサム,73歳 (男),新人,無所属,江戸川区議会議員（10期40年）、無職,M


## 1.3 Clean scraped data

In [24]:
# merge assembly and mayor candidates dataframes
candidates_df = pd.concat([m_candidates_df, a_candidates_df], ignore_index=True)

In [25]:
# clean "name" column
candidates_df['name_cleaned'] = candidates_df.apply(
    lambda x: x['name'].strip('\n').strip('\t').replace(x['katakana'], ""), axis=1
)
candidates_df = candidates_df.drop("name", axis=1).rename(columns={'name_cleaned': 'name'})

In [26]:
# create "age" column from "age_sex"
candidates_df['age'] = candidates_df.age_sex.apply(lambda x: re.findall(r".+歳", x)[0].replace('歳', "").replace("-", "NA"))

In [27]:
# create "sex" column from "age_sex"
candidates_df['sex'] = candidates_df.age_sex.apply(lambda x: re.findall(r"\(([^)]*)\)", x)) 
candidates_df['sex'] = [item[0] if len(item) > 0 else "NA" for item in candidates_df.sex]
candidates_df['sex'] = np.where(
    candidates_df.sex == "男", "M", np.where(
        candidates_df.sex == "女", "F", np.where(
            candidates_df.sex == "", "NA", "NA"
            )
        )
    )

In [28]:
# remove "age_sex" column
candidates_df = candidates_df.drop("age_sex", axis=1)

In [5]:
candidates_df.party = np.where(candidates_df.party == "NHK党", "政治家女子48党", candidates_df.party)

In [ ]:
party_dict = {
    '自由民主党': 'LDP',
    '公明党': 'NKP',
    '無所属': 'Unaffiliated',
    '日本共産党': 'JCP',
    '立憲民主党': 'CDP',
    '政治家女子48党': 'Joshi',
    '日本維新の会': 'Ishin',
    '江戸川・生活者ネットワーク': 'Edogawa Seikatsusha Network',
    'あたらしい党': 'Atarashii',
    'れいわ新選組': 'Reiwa',
    '参政党': 'Sansei',
    '国民民主党': 'DPP',
    '目覚めよ日本党': 'Mezameyo Nihon'
    }

In [ ]:
candidates_df['party_en'] = candidates_df.party.map(party_dict)

# 2. Search Twitter accounts of the candidates

1. Manually searched a candidate's name on Twitter
2. If not found, searched the name + "江戸川" (Edogawa) on Twitter
3. If not found, searched the name + "江戸川" on Google. Look for Twitter account in the result. If Twitter account is not found but their website/blog etc. is found, looked for Twitter id on it
4. If still not found, `"NA"` 

In [43]:
twitter_ids = [
    '3sedogawa', 'jcpedo_sbata', 
    'kanai_jsc', 'maruyamareiko3', 'edogawafuture', 'officefukumoto', 'Iwata_Masakazu', 
    'yw8pl21', 'masao_igarashi', 'Hadano_Yuji', 'mamiyayumi', 'NA',
    'sjj48_honoka', 'ken16000', 'NA', 'NA', 'NA', 
    'NA', 'NA', 'omiram33', 'NA', 'hayashiakiko_', 
    'kobayashitomoo_', 'Henry66239746', 'TakizawaYasuko', 'sasayan210', 'NA', 
    'NA', 'nozakimakoto77', 'NA', 'tanakajuichi', 'hayakawa_kazue', 
    'NA', 'NA', 'kazu3816', 'edogawasasaki', 'kometokoro', 
    'iYWWp826zmWNf7g', 'K328n', '59Hori', 'sekine_komei', 'W6B7M4s57S9JG4I',
    'TakasiNakamichi', 'NA', 'NA', 'nanapy_1987', 'kimotomayu44', 
    'NA', 'komei_kawai', 'ryoichiezoe', 'edoken', 'ReiwaTamura',
    'omtanoriko', 'NA', 'kenji_makino', 'jcp_oaya', 'Pecchan_CHANNEL',
    'NA'
    ]

In [44]:
candidates_df['twitter_id'] = twitter_ids

In [45]:
candidates_df

,katakana,new_old,party,vocation,category,name,age,sex,twitter_id
0,サイトウ タケシ,現職,無所属,江戸川区長,M,斉藤 たけし,60,M,3sedogawa
1,セバタ イサム,新人,無所属,江戸川区議会議員（10期40年）、無職,M,セバタ 勇,73,M,jcpedo_sbata
2,カナイ タカシ,新人,自由民主党,弁護士,A,金井 たかし,60,M,kanai_jsc
3,マルヤマ レイコ,新人,日本維新の会,政治家秘書,A,丸山 れいこ,70,F,maruyamareiko3
4,オオタ アツシ,新人,無所属,会社員,A,太田 あつし,48,M,edogawafuture
5,フクモト ミツヒロ,現職,自由民主党,江戸川区議会議長,A,福本 みつひろ,51,M,officefukumoto
6,イワタ マサカズ,現職,無所属,江戸川区議会・区議会江戸川クラブ（幹事長）,A,岩田 まさかず,49,M,Iwata_Masakazu
7,カツヤマ マユミ,新人,自由民主党,自由民主党東京都江戸川区第十一支部長,A,勝山 まゆみ,54,F,yw8pl21
8,イガラシ マサオ,新人,参政党,NA,A,五十嵐 まさお,38,M,masao_igarashi
9,ハダノ ユウジ,新人,自由民主党,会社員,A,肌野 ゆうじ,53,M,Hadano_Yuji


In [7]:
# save candidates df
candidates_df.to_pickle("data/candidates_df.p")

In [47]:
# write twitter ids to .txt file
with open('data/ids.txt', "w") as txtfile:
    txtfile.write('\n'.join(id for id in candidates_df.twitter_id if id != "NA"))

# 3. Get tweets
- Use <a href="https://github.com/marquisvictor/Optimized-Modified-GetOldTweets3-OMGOT">`Get Old Tweets 3`</a> library to collect data - use `helpers/script.py` to write `helpers/commands.sh` and run `helpers/commands.sh` (`$ bash helpers/commands.sh`). This saves one csv file per twitter id (`twitter_id.csv`) in `data/raw` directory
- Scope of analysis: After last local election (21 Apr 2019) ~ 19 Apr 2023